In [ ]:
from langchain_groq import ChatGroq

from langgraph.graph import StateGraph,START,END

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from pydantic import BaseModel,Field
from typing import Annotated
# from langgraph.graph.message import BaseMessage

# import json


In [2]:
class JobData(BaseModel):
    user_info : str = Field(default= "I am Aman Prajapat interested in AI field and have good skill set in the respective field",        
                                     description="Instruction for creating a proper message")
    subject : str = Field(default="")
    body : str  = Field(default="")
    email :str = Field(default="")
    post_data :str = Field(default="")
    feedback :str = Field(default="No feedback")



In [3]:

post_text = """🚀 We’re Hiring! Freshers Welcome! 🚀

Are you a recent graduate ready to kick-start your career?
We’re looking for enthusiastic and passionate freshers to join our growing team at QSS Technosoft!

🔹 Position: Software Engineer
🔹 Location: Noida
🔹 Experience: 0–1 year (Freshers encouraged to apply)

What we’re looking for:
✅ A strong willingness to learn and grow
✅ Good communication and problem-solving skills
✅ A positive attitude and team spirit
✅ Basic knowledge of relevant tools/skills

What we offer:
🌱 Mentorship and hands-on training
🚀 Growth opportunities in a dynamic work environment
🤝 A collaborative, inclusive company culture
💼 Competitive salary and benefits

If you’re eager to begin your professional journey with a company that values learning and innovation — we’d love to hear from you!
📩 Apply now by sending your resume to ayush.srivastava@qsstechnosoft.com
"""

# create the JobData object with only instruction and post_data
job = JobData(
    instrctions="You are an expert in writing short and concise emails. Keep subject punchy and body ~3-5 short sentences.",
    post_data=post_text
)



In [4]:
model = ChatGroq(model= "openai/gpt-oss-120b",temperature=0.3)

In [5]:
# email extractor
def extract_mail_id(state:JobData):
    prompt = PromptTemplate(
        template= "From the text below, extract ONLY the email address  where the candidate has to apply.  Return ONLY the email (no other text).\n\n Post content:\n{post_data}"
    )
    mail_chain = prompt | model | StrOutputParser()

    email = mail_chain.invoke({'post_data':state.post_data})

    return {'email':email}

In [6]:
extract_mail_id(job)

{'email': 'ayush.srivastava@qsstechnosoft.com'}

In [7]:

def create_mail(state: JobData):
    prompt = PromptTemplate(
        template=(
            "You are an expert email writer.\n"
            "Use the following details to create a professional job application email.\n\n"
            "The email should be short and sweet with no place holders because its a automation system for send mail.\n"
            "User information:\n{user_information}\n\n"
            "Job Post Content:\n{post_data}\n\n"
            "Now generate the email in JSON format as shown:\n"
            "{{\n"
            '  "subject": "Your email subject",\n'
            '  "body": "Your email body with proper formating"\n'
            "}}"
        ),
        input_variables=["user_information", "feedback", "post_data"]
    )

    mail_chain = prompt | model | JsonOutputParser()

    # Step 3: Invoke model
    response = mail_chain.invoke({
        "user_information": state.user_info,
        "post_data": state.post_data
    })


    # try:
    #     data = json.loads(response)
    # except json.JSONDecodeError:
    #     # Fallback: create empty keys if format isn’t valid JSON
    #     data = {"subject": "", "body": response}
    # return response
    return {'body':response["body"],'subject':response['subject']}


In [8]:
res = create_mail(job)
print(res['subject'])
print(res['body'])

Application for Software Engineer – Aman Prajapat
Dear Mr. Srivastava,

I am writing to express my interest in the Software Engineer position at QSS Technosoft (Noida). I have recently graduated and am eager to start my career in the AI field. My academic projects and self‑learning have equipped me with a solid foundation in machine learning concepts, Python programming, and data analysis tools.

I am a quick learner, enjoy problem‑solving, and thrive in collaborative environments. I am confident that my enthusiasm and technical background align well with the mentorship and growth opportunities your team offers.

Please find my resume attached for your review. I look forward to the possibility of contributing to QSS Technosoft and growing alongside your talented team.

Thank you for considering my application.

Best regards,
Aman Prajapat
Phone: +91-XXXXXXXXXX
Email: aman.prajapat@example.com


In [ ]:
def update_mail(state: JobData):
    prompt = PromptTemplate(
        template=(
            "You are an expert email rewriter.\n"
            "Your task is to update the existing email according to the user's feedback. "
            "The email should be short and sweet with no place holders because its a automation system for send mail.\n"
            "### Feedback :\n{feedback}\n\n"
            "### Previous Email:\n"
            "Subject: {subject}\n"
            "Body: {body}\n\n"
            "Now rewrite the email according to the feedback. "
            "Return the response in JSON format as follows:\n"
            "{{\n"
            '  "subject": "Updated subject",\n'
            '  "body": "Updated body"\n'
            "}}"
        ),
        input_variables=["feedback", "subject", "body"]
    )

    # Step 2: Build LLM chain
    update_chain = prompt | model | JsonOutputParser()

    # Step 3: Generate updated mail
    response = update_chain.invoke({
        "feedback": state.feedback,
        "subject": state.subject,
        "body": state.body
    })


    return {
        "subject": response['subject'],
        "body": response['body']
    }


In [10]:
job.feedback = "just add one line in which i showed that i have done two internship in the ai field in which craeted chatbot and also fine tuned a llm and deployed the company projects"

In [11]:
job.feedback

'just add one line in which i showed that i have done two internship in the ai field in which craeted chatbot and also fine tuned a llm and deployed the company projects'

In [12]:
data = update_mail(job)
print(res['subject'])
print(res['body'])

Application for Software Engineer – Aman Prajapat
Dear Mr. Srivastava,

I am writing to express my interest in the Software Engineer position at QSS Technosoft (Noida). I have recently graduated and am eager to start my career in the AI field. My academic projects and self‑learning have equipped me with a solid foundation in machine learning concepts, Python programming, and data analysis tools.

I am a quick learner, enjoy problem‑solving, and thrive in collaborative environments. I am confident that my enthusiasm and technical background align well with the mentorship and growth opportunities your team offers.

Please find my resume attached for your review. I look forward to the possibility of contributing to QSS Technosoft and growing alongside your talented team.

Thank you for considering my application.

Best regards,
Aman Prajapat
Phone: +91-XXXXXXXXXX
Email: aman.prajapat@example.com


## LangGraph Part

In [13]:
applyGraph = StateGraph(JobData)

In [ ]:
def wantToupdateMail(state:JobData):
    condition = input("want to update the mail")

    if condition== "yes":
        state.feedback = input("write your feedback")
        return "update_mail"
    else:
        return "send_email"

In [ ]:
def send_mail(state:JobData):
    pass

In [ ]:
applyGraph.add_node("input")
applyGraph.add_node("find_email")
applyGraph.add_node("gen_body_sub")
applyGraph.add_node("user_feedback")
applyGraph.add_node("send_mail")
applyGraph.add_node("update_message")


applyGraph.add_edge(START,"input")
applyGraph.add_edge("input","find_email")
applyGraph.add_edge("input","gen_body_sub")
applyGraph.add_conditional_edges("gen_body_sub",wantToupdateMail,{"update_mail":update_mail,"send_email":send_mail})
applyGraph.add_edge("update_mail",)
applyGraph.add_edge("send_mail",END)